In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('../data/cleaned_error_analysis.csv')
df = pd.read_excel('../data/cleaned_error_analysis_v2.xlsx')
df

,Unnamed: 0,pmid,start,end,tag,text,source,correct,error_type,annotator,correction,remarks,confidence level (1-5),"**1 is not confident at all, 5 is absolutely confident**"
0,0,58651,26,42,Group Characteristic,renal transplant,model,y,NaN,ccurtis,NaN,NaN,NaN,NaN
1,1,58651,144,149,Drug Intervention,HAHTG,human,y,NaN,ccurtis,NaN,NaN,NaN,NaN
2,2,58651,165,175,Non-Study Drug,prednisone,human,y,wrong label,ccurtis,NaN,NaN,NaN,NaN
3,3,58651,180,192,Non-Study Drug,azathioprine,human,y,wrong label,ccurtis,NaN,NaN,NaN,NaN
4,4,58651,194,200,Drug Intervention,lmuran,model,n,wrong label,ccurtis,Non-Study Drug,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5561,5561,36206137,2226,2228,Quantitative Measurement,53,human,n,too little,eduvaris,0.53,NaN,5.0,NaN
5562,5562,36206137,2226,2230,Quantitative Measurement,53â€Š%,model,y,NaN,eduvaris,NaN,NaN,NaN,NaN
5563,5563,36206137,2235,2237,Quantitative Measurement,47,human,n,too little,eduvaris,0.47,NaN,5.0,NaN
5564,5564,36206137,2235,2239,Quantitative Measurement,47â€Š%,model,y,NaN,eduvaris,NaN,NaN,NaN,NaN


Total Number of tagged mentions

In [5]:
df_model = df[df['source'] == 'model']
df_human = df[df['source'] == 'human']
print("Number of model tagged mentions :", len(df_model))
print("Number of human tagged mentions :", len(df_human))

Number of model tagged mentions : 3103
Number of human tagged mentions : 2463


Number of pmids

In [6]:
model_unique_pmids = df_model['pmid'].nunique()
human_unique_pmids = df_human['pmid'].nunique()
print(f"Model unique PMIDs: {model_unique_pmids}")
print(f"Human unique PMIDs: {human_unique_pmids}")

Model unique PMIDs: 237
Human unique PMIDs: 236


Different pmids

In [7]:
pmid_difference1 = set(df_model['pmid']) - set(df_human['pmid'])
pmid_difference2 = set(df_human['pmid']) - set(df_model['pmid'])
pmid_difference = pmid_difference1.union(pmid_difference2)
print("The pmids that are not in common are :", pmid_difference)
print("Model was evaluated on", pmid_difference1, " that human was not evaluated on")
print("Human was evaluated on", pmid_difference2, " that model was not evaluated on")

The pmids that are not in common are : {8151609, 2719459, 3582605}
Model was evaluated on {8151609, 3582605}  that human was not evaluated on
Human was evaluated on {2719459}  that model was not evaluated on


Different error types

In [8]:
set(df_model['error_type'])

{'multiple entities',
 nan,
 'not capturable',
 'not needed',
 'too little',
 'too little or too much based on context',
 'too much',
 'wrong label'}

Number of correctly labeled mentions

In [9]:
model_correct_label_count = df_model['error_type'].isna().sum() - (df_model[(df_model['correct'] == 'n') & (df_model['error_type'].isna())].shape[0])
human_correct_label_count = df_human['error_type'].isna().sum() - (df_human[(df_human['correct'] == 'n') & (df_human['error_type'].isna())].shape[0])
print(f"Number of correctly labeled mentions by the model: {model_correct_label_count}, {model_correct_label_count/len(df_model)*100}%")
print(f"Number of correctly labeled mentions by the human: {human_correct_label_count}, {human_correct_label_count/len(df_human)*100}%")

Number of correctly labeled mentions by the model: 2218, 71.47921366419594%
Number of correctly labeled mentions by the human: 1240, 50.345107592367036%


Number of uncorrectly labeled mentions

In [10]:
model_uncorrect_label_count = df_model['error_type'].notna().sum() + (df_model[(df_model['correct'] == 'n') & (df_model['error_type'].isna())].shape[0])
human_uncorrect_label_count = df_human['error_type'].notna().sum() + (df_human[(df_human['correct'] == 'n') & (df_human['error_type'].isna())].shape[0])
print(f"Number of correctly tagged mentions but uncorrectly labeled by the model: {model_uncorrect_label_count}, {model_uncorrect_label_count/len(df_model)*100}%")
print(f"Number of correctly tagged mentions but uncorrectly labeled by the human: {human_uncorrect_label_count}, {human_uncorrect_label_count/len(df_human)*100}%")

Number of correctly tagged mentions but uncorrectly labeled by the model: 885, 28.52078633580406%
Number of correctly tagged mentions but uncorrectly labeled by the human: 1223, 49.654892407632964%


Type of error : wrong label / too much / too little / multiple entities / not needed (= shouldn't have been tagged)

In [11]:
model_wrong_label_count = df_model[df_model['error_type'] == 'wrong label'].shape[0]
human_wrong_label_count = df_human[df_human['error_type'] == 'wrong label'].shape[0]
print(f"Number of mentions that got 'wrong label' from the model: {model_wrong_label_count}, {model_wrong_label_count/model_uncorrect_label_count*100}%")
print(f"Number of mentions that got 'wrong label' from the human: {human_wrong_label_count}, {human_wrong_label_count/human_uncorrect_label_count*100}%")

model_too_little_count = df_model[df_model['error_type'] == 'too little'].shape[0]
human_too_little_count = df_human[df_human['error_type'] == 'too little'].shape[0]
print(f"Number of mentions that got 'too little' from the model: {model_too_little_count}, {model_too_little_count/model_uncorrect_label_count*100}%")
print(f"Number of mentions that got 'too little' from the human: {human_too_little_count}, {human_too_little_count/human_uncorrect_label_count*100}%")

model_not_needed_tag_count = df_model[(df_model['error_type'] == 'not needed')].shape[0]
human_not_needed_tag_count = df_human[(df_human['error_type'] == 'not needed')].shape[0]
print(f"Number of mentions that got 'not needed' by the model: {model_not_needed_tag_count}, {model_not_needed_tag_count/model_uncorrect_label_count*100}%")
print(f"Number of mentions that got 'not needed' by the human: {human_not_needed_tag_count}, {human_not_needed_tag_count/human_uncorrect_label_count*100}%")

model_too_much_count = df_model[df_model['error_type'] == 'too much'].shape[0]
human_too_much_count = df_human[df_human['error_type'] == 'too much'].shape[0]
print(f"Number of mentions that got 'too much' from the model: {model_too_much_count}, {model_too_much_count/model_uncorrect_label_count*100}%")
print(f"Number of mentions that got 'too much' from the human: {human_too_much_count}, {human_too_much_count/human_uncorrect_label_count*100}%")

model_multiple_entities_count = df_model[df_model['error_type'] == 'multiple entities'].shape[0]
human_multiple_entities_count = df_human[df_human['error_type'] == 'multiple entities'].shape[0]
print(f"Number of mentions that got 'multiple entities' from the model: {model_multiple_entities_count}, {model_multiple_entities_count/model_uncorrect_label_count*100}%")
print(f"Number of mentions that got 'multiple entities' from the human: {human_multiple_entities_count}, {human_multiple_entities_count/human_uncorrect_label_count*100}%")

model_little_much_count = df_model[df_model['error_type'] == 'too little or too much based on context'].shape[0]
human_little_much_count = df_human[df_human['error_type'] == 'too little or too much based on context'].shape[0]
print(f"Number of mentions that got 'too little or too much based on context' from the model: {model_little_much_count}, {model_little_much_count/model_uncorrect_label_count*100}%")
print(f"Number of mentions that got 'too little or too much based on context' from the human: {human_little_much_count}, {human_little_much_count/human_uncorrect_label_count*100}%")

model_empty_count = (df_model[(df_model['correct'] == 'n') & (df_model['error_type'].isna())].shape[0])
human_empty_count = (df_human[(df_human['correct'] == 'n') & (df_human['error_type'].isna())].shape[0])
print(f"Number of mentions that shouldn't have been tagged by the model: {model_empty_count}, {model_empty_count/model_uncorrect_label_count*100}%")
print(f"Number of mentions that shouldn't have been tagged by the human: {human_empty_count}, {human_empty_count/human_uncorrect_label_count*100}%")

Number of mentions that got 'wrong label' from the model: 302, 34.12429378531073%
Number of mentions that got 'wrong label' from the human: 481, 39.329517579721994%
Number of mentions that got 'too little' from the model: 239, 27.005649717514125%
Number of mentions that got 'too little' from the human: 279, 22.812755519215045%
Number of mentions that got 'not needed' by the model: 141, 15.932203389830507%
Number of mentions that got 'not needed' by the human: 129, 10.54783319705642%
Number of mentions that got 'too much' from the model: 129, 14.576271186440678%
Number of mentions that got 'too much' from the human: 219, 17.906786590351594%
Number of mentions that got 'multiple entities' from the model: 41, 4.632768361581921%
Number of mentions that got 'multiple entities' from the human: 79, 6.459525756336877%
Number of mentions that got 'too little or too much based on context' from the model: 14, 1.5819209039548021%
Number of mentions that got 'too little or too much based on context

In [13]:
print(df_model['error_type'].value_counts().sum())
df_model['error_type'].value_counts()

867


error_type
wrong label                                302
too little                                 239
not needed                                 141
too much                                   129
multiple entities                           41
too little or too much based on context     14
not capturable                               1
Name: count, dtype: int64

In [14]:
df_human['error_type'].value_counts()

error_type
wrong label                                       481
too little                                        279
too much                                          219
not needed                                        129
multiple entities                                  79
too little or too much based on context             8
too much or multiple entities based on context      3
not necessary/wrong label                           2
Name: count, dtype: int64